In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset=pd.read_csv('Google_Stock_Price_Train.csv');
train_set=dataset.iloc[:,1:2].values

In [ ]:
print(train_set)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
train_set_scaled=sc.fit_transform(train_set)

In [ ]:
train_set_scaled.shape[0]

1258

In [ ]:
X_train=[]
y_train=[]
for i in range(60,train_set_scaled.shape[0]):
  X_train.append(train_set_scaled[i-60:i,0])
  y_train.append(train_set_scaled[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)


In [ ]:
X_train

In [ ]:
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
print(X_train)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regressor=Sequential()

In [ ]:
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))

In [ ]:
regressor.add(Dropout(0.2))

In [ ]:
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

In [ ]:
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

In [ ]:
regressor.add(LSTM(units=50,return_sequences=False))
regressor.add(Dropout(0.2))

In [ ]:
regressor.add(Dense(units=1))

In [ ]:
regressor.compile(optimizer="adam",loss="mean_squared_error")

In [ ]:
regressor.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
38/38 [==============================] - 11s 113ms/step - loss: 0.0368
Epoch 2/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0058
Epoch 3/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0054
Epoch 4/100
38/38 [==============================] - 4s 112ms/step - loss: 0.0068
Epoch 5/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0052
Epoch 6/100
38/38 [==============================] - 4s 112ms/step - loss: 0.0052
Epoch 7/100
38/38 [==============================] - 4s 112ms/step - loss: 0.0047
Epoch 8/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0046
Epoch 9/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0047
Epoch 10/100
38/38 [==============================] - 4s 115ms/step - loss: 0.0042
Epoch 11/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0044
Epoch 12/100
38/38 [==============================] - 4s 114ms/step - loss: 0.0049
Epoch 13/100

In [ ]:
dataset_test=pd.read_csv('Google_Stock_Price_Test.csv');
test_set=dataset_test.iloc[:,1:2].values

In [ ]:
total_dataset=pd.concat((dataset['Open'],dataset_test['Open']),axis=0)

In [ ]:
input=total_dataset[(len(total_dataset)-len(dataset_test))-60:].values

In [ ]:
input=input.reshape(-1,1)

In [ ]:
input=sc.transform(input)

In [ ]:
X_test=[]
for i in range(60,input.shape[0]):
  X_test.append(input[i-60:i,0])
X_test=np.array(X_test)

In [ ]:
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
predicted_stock_price=regressor.predict(X_test)

In [ ]:
predicted_stock_price=sc.inverse_transform(predicted_stock_price)

In [ ]:
predicted_stock_price

array([[774.52106],
       [771.7064 ],
       [771.803  ],
       [773.06   ],
       [776.2085 ],
       [781.8028 ],
       [786.87604],
       [788.7969 ],
       [788.92676],
       [788.59143],
       [788.3859 ],
       [788.2782 ],
       [788.30426],
       [789.01294],
       [790.0887 ],
       [794.6364 ],
       [801.3338 ],
       [808.3887 ],
       [812.01514],
       [807.732  ]], dtype=float32)

In [ ]:
plt.plot(predicted_stock_price,color='blue',label='predicted')
plt.plot(test_set,color='red',label='real')
plt.xlabel('time')
plt.ylabel('stock prices')
plt.title("stock prediction")
plt.legend
plt.show